# Revisiting the NYC Taxi DataSet Model Architecture Part 4

In this notebook, the model architecture is being changed in order to understand the correlation between the model architecture and the prediction quality.
This model uses only the end time as input.
A Dense Layer follows. Its output goes into another Dense Layer with SoftMax activation to obtain the logits.
The deep learning problem can thus be described as follows:
Predict a location based on the current time.


In [1]:
import numpy as np
np.random.seed(0)
import tensorflow as tf
import pandas as pd
from tensorflow import feature_column
from tensorflow.keras import layers
import import_ipynb

In [2]:
from model_helper import ModelHelper

importing Jupyter notebook from model_helper.ipynb


# Dataset

In [3]:
df = pd.read_csv("./ma_results/trips_with_zones_final.csv")
df = df.head(10000000)
df.head(10)

,medallion,pickup_week_day,pickup_hour,pickup_day,pickup_month,dropoff_week_day,dropoff_hour,dropoff_day,dropoff_month,pickup_location_id,dropoff_location_id
0,00005007A9F30E289E760362F69E4EAD,1,0,1,1,1,0,1,1,162.0,262.0
1,00005007A9F30E289E760362F69E4EAD,1,0,1,1,1,0,1,1,262.0,239.0
2,00005007A9F30E289E760362F69E4EAD,1,0,1,1,1,1,1,1,239.0,236.0
3,00005007A9F30E289E760362F69E4EAD,1,1,1,1,1,1,1,1,236.0,41.0
4,00005007A9F30E289E760362F69E4EAD,1,1,1,1,1,1,1,1,41.0,211.0
5,00005007A9F30E289E760362F69E4EAD,1,1,1,1,1,2,1,1,211.0,238.0
6,00005007A9F30E289E760362F69E4EAD,1,2,1,1,1,2,1,1,238.0,142.0
7,00005007A9F30E289E760362F69E4EAD,1,2,1,1,1,2,1,1,142.0,263.0
8,00005007A9F30E289E760362F69E4EAD,1,2,1,1,1,3,1,1,263.0,48.0
9,00005007A9F30E289E760362F69E4EAD,1,3,1,1,1,3,1,1,48.0,246.0


In [4]:
# Check dtypes of the attributes
df.dtypes

medallion               object
pickup_week_day          int64
pickup_hour              int64
pickup_day               int64
pickup_month             int64
dropoff_week_day         int64
dropoff_hour             int64
dropoff_day              int64
dropoff_month            int64
pickup_location_id     float64
dropoff_location_id    float64
dtype: object

In [5]:
# Drop the medallion, it is not needed for this example
df.drop(['medallion'], axis=1, inplace=True)

Because there are too many taxis (over 9000) it is better to take the 100 taxi with the major number of records

In [6]:
# Cast the columns type to int32
dictionary = {'pickup_week_day': 'int32', 'pickup_hour': 'int32', 'pickup_day': 'int32', 'pickup_month': 'int32', 'dropoff_week_day': 'int32', 'dropoff_hour': 'int32', 'dropoff_day': 'int32', 'dropoff_month': 'int32', 'pickup_location_id':'int32', 'dropoff_location_id':'int32'}
df = df.astype(dictionary, copy=True)
df.dtypes

pickup_week_day        int32
pickup_hour            int32
pickup_day             int32
pickup_month           int32
dropoff_week_day       int32
dropoff_hour           int32
dropoff_day            int32
dropoff_month          int32
pickup_location_id     int32
dropoff_location_id    int32
dtype: object

We can use the other taxis to create a local test and validation sets

Now we need to create the location sequence for each user

In [7]:
mh = ModelHelper(df, 129)

In [8]:
# Call the function
mh.df_to_location_sequence()

print(mh.df)

            index  location_id  day  month  hour_sin      hour_cos  \
0               0          162    1      1  0.000000  1.000000e+00   
1              12          230    1      1  0.707107  7.071068e-01   
2              13          125    1      1  0.707107  7.071068e-01   
3              15           48    1      1  0.866025  5.000000e-01   
4              18          170    1      1  1.000000  6.123234e-17   
...           ...          ...  ...    ...       ...           ...   
13731996  7284341          161   26      1 -0.500000 -8.660254e-01   
13731997  7284341          161   26      1 -0.500000 -8.660254e-01   
13731998  7284342          132   26      1 -0.707107 -7.071068e-01   
13731999  7284343          141   26      1 -0.866025 -5.000000e-01   
13732000  7284344          141   26      1 -0.866025 -5.000000e-01   

          week_day_sin  week_day_cos  weekend  
0             0.781831      0.623490        0  
1             0.781831      0.623490        0  
2             0

In [9]:
mh.train_val_test_split()
print(len(mh.df_train), 'train examples')
print(len(mh.df_val), 'validation examples')
print(len(mh.df_test), 'test examples')

8788480 train examples
2197120 validation examples
2746401 test examples


In [10]:
mh.split_data()
mh.list_test[0]

,index,location_id,day,month,hour_sin,hour_cos,week_day_sin,week_day_cos,weekend
10985600,5283998,246,4,1,-0.866025,0.500000,-0.433884,-0.900969,0
10985601,5283999,107,4,1,-0.707107,0.707107,-0.433884,-0.900969,0
10985602,5284000,142,4,1,-0.707107,0.707107,-0.433884,-0.900969,0
10985603,5284001,48,4,1,-0.500000,0.866025,-0.433884,-0.900969,0
10985604,5284001,48,4,1,-0.500000,0.866025,-0.433884,-0.900969,0
...,...,...,...,...,...,...,...,...,...
10985724,5284091,234,7,1,0.000000,1.000000,0.000000,1.000000,0
10985725,5284092,162,7,1,0.258819,0.965926,0.000000,1.000000,0
10985726,5284093,142,7,1,0.500000,0.866025,0.000000,1.000000,0
10985727,5284093,142,7,1,0.500000,0.866025,0.000000,1.000000,0


In [11]:
mh.set_batch_size(128)
mh.create_batch_dataset()
mh.test_dataset

<BatchDataset shapes: ({start_place: (128, 128), start_hour_sin: (128, 128), start_hour_cos: (128, 128), weekend: (128, 128), week_day_sin: (128, 128), week_day_cos: (128, 128), end_hour_sin: (128,), end_hour_cos: (128,), end_weekend: (128,), end_week_day_sin: (128,), end_week_day_cos: (128,)}, (128,)), types: ({start_place: tf.int32, start_hour_sin: tf.float64, start_hour_cos: tf.float64, weekend: tf.int32, week_day_sin: tf.float64, week_day_cos: tf.float64, end_hour_sin: tf.float64, end_hour_cos: tf.float64, end_weekend: tf.int32, end_week_day_sin: tf.float64, end_week_day_cos: tf.float64}, tf.int32)>

In [12]:
mh.set_target_column_name('location_id')
mh.set_vocab_size()
mh.set_numerical_column_names(['start_hour_sin', 'start_hour_cos', 'weekend', 'week_day'])

In [13]:
# Number of RNN units
dense_units = 512

# Create a model
def create_model():
  N = mh.total_window_length
  batch_size = mh.batch_size
  number_of_places = mh.vocab_size

  # Shortcut to the layers package
  l = tf.keras.layers


  other_feature_inputs = {
    'end_hour_sin': tf.keras.Input((1, ), batch_size=batch_size, name='end_hour_sin'),
    'end_hour_cos': tf.keras.Input((1, ), batch_size=batch_size, name='end_hour_cos'),
    'end_weekend': tf.keras.Input((1, ), batch_size=batch_size, name='end_weekend'),
    'end_week_day_sin': tf.keras.Input((1, ), batch_size=batch_size, name='end_week_day_sin'),
    'end_week_day_cos': tf.keras.Input((1, ), batch_size=batch_size, name='end_week_day_cos')
  }


  end_hour_sin = feature_column.numeric_column("end_hour_sin", shape=(1))
  end_hour_sin_feature = l.DenseFeatures(end_hour_sin)(other_feature_inputs)

  end_hour_cos = feature_column.numeric_column("end_hour_cos", shape=(1))
  end_hour_cos_feature = l.DenseFeatures(end_hour_cos)(other_feature_inputs)

  end_weekend = feature_column.numeric_column("end_weekend", shape=(1))
  end_weekend_feature = l.DenseFeatures(end_weekend)(other_feature_inputs)

  end_week_day_sin = feature_column.numeric_column("end_week_day_sin", shape=(1))
  end_week_day_sin_feature = l.DenseFeatures(end_week_day_sin)(other_feature_inputs)

  end_week_day_cos = feature_column.numeric_column("end_week_day_cos", shape=(1))
  end_week_day_cos_feature = l.DenseFeatures(end_week_day_cos)(other_feature_inputs)


  concatenate_2 = l.Concatenate(axis=1)([end_hour_sin_feature, end_hour_cos_feature, end_weekend_feature, end_week_day_sin_feature, end_week_day_cos_feature])

  dense_0 = layers.Dense(512)(concatenate_2)

  # Last layer with an output for each places
  dense_1 = layers.Dense(number_of_places)(dense_0)

  # Softmax output layer
  output = l.Softmax()(dense_1)

  # To return the Model, we need to define its inputs and outputs
  # In out case, we need to list all the input layers we have defined
  inputs = list(other_feature_inputs.values())

	# Return the Model
  return tf.keras.Model(inputs=inputs, outputs=output)

In [14]:
# Get the model and compile it
mh.assign_model(create_model())
mh.compile_model()

# Training

In [15]:
mh.set_num_epochs(10)
mh.fit_model()

Epoch 1/10


C:\Users\WoodPecker\Documents\Privat\HTW-Master\Sem3\PA\HumanMobilityPredictionMA\venv\lib\site-packages\tensorflow\python\keras\engine\functional.py:543: UserWarning: Input dict contained keys ['start_place', 'start_hour_sin', 'start_hour_cos', 'weekend', 'week_day_sin', 'week_day_cos'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


532/532 [==============================] - 2s 5ms/step - loss: 4.2560 - sparse_categorical_accuracy: 0.0447 - val_loss: 4.1679 - val_sparse_categorical_accuracy: 0.0459
Epoch 2/10
532/532 [==============================] - 2s 4ms/step - loss: 4.1348 - sparse_categorical_accuracy: 0.0472 - val_loss: 4.1658 - val_sparse_categorical_accuracy: 0.0458
Epoch 3/10
532/532 [==============================] - 2s 4ms/step - loss: 4.1319 - sparse_categorical_accuracy: 0.0474 - val_loss: 4.1655 - val_sparse_categorical_accuracy: 0.0459
Epoch 4/10
532/532 [==============================] - 2s 4ms/step - loss: 4.1303 - sparse_categorical_accuracy: 0.0474 - val_loss: 4.1654 - val_sparse_categorical_accuracy: 0.0460
Epoch 5/10
532/532 [==============================] - 2s 4ms/step - loss: 4.1292 - sparse_categorical_accuracy: 0.0473 - val_loss: 4.1655 - val_sparse_categorical_accuracy: 0.0461
Epoch 6/10
532/532 [==============================] - 2s 4ms/step - loss: 4.1283 - sparse_categorical_accuracy:

# Evaluation

In [16]:
mh.evaluate_model()

166/166 [==============================] - 0s 3ms/step - loss: 4.1645 - sparse_categorical_accuracy: 0.0476


In [17]:
mh.model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
end_hour_cos (InputLayer)       [(128, 1)]           0                                            
__________________________________________________________________________________________________
end_hour_sin (InputLayer)       [(128, 1)]           0                                            
__________________________________________________________________________________________________
end_week_day_cos (InputLayer)   [(128, 1)]           0                                            
__________________________________________________________________________________________________
end_week_day_sin (InputLayer)   [(128, 1)]           0                                            
_______________________________________________________________________________________

In [18]:
mh.print_test_prediction_info()

logits
Shape :  (21248, 264)
Example [0] :  [3.73983220e-03 2.88076350e-04 2.09020095e-06 4.98625704e-05
 6.20020088e-03 1.72556338e-06 5.80900542e-06 1.04348678e-02
 2.75753846e-05 4.79899958e-04 3.14354576e-04 8.13703155e-05
 3.29892064e-04 1.15144160e-02 2.52301502e-03 2.58236367e-04
 8.16771862e-05 4.09002742e-03 3.08402465e-04 1.30072613e-06
 1.80587685e-05 6.31607836e-05 4.38195595e-04 3.89668043e-04
 3.15937167e-03 4.58362186e-03 1.90445076e-04 1.83375460e-06
 1.41122699e-04 6.30446710e-04 1.81630116e-06 7.19545205e-05
 1.30022643e-04 3.05972854e-03 7.10057619e-04 5.10225909e-05
 1.68573984e-03 8.60213116e-03 6.29741271e-06 2.55379156e-04
 2.89552403e-03 6.03324594e-03 6.02895487e-03 3.91792040e-03
 3.14773797e-06 4.74546617e-03 6.27149420e-05 8.04327938e-05
 3.13153751e-02 5.59362909e-03 1.12483427e-02 8.62848392e-05
 1.24995434e-03 1.99395421e-04 9.17893602e-04 4.94053005e-04
 3.38320300e-04 1.79650544e-06 3.61860407e-06 1.70014641e-06
 6.06207359e-05 4.81992541e-03 1.29526283

Only using the time components results in a sub 5% prediction accuracy.
This confirms the conclusions from the past model architecture analysis.